In [ ]:
import FMR
import numpy as np
import matplotlib.pyplot as plt
import glob,os

In [ ]:
#directory = '/Volumes/Transcend/Suzuki/Data/ppblsmo/FMR/ppblsmo18A-IP110/'
#olddir = os.getcwd()
#os.chdir(directory)
filenames = glob.glob('*.csv')
for f in filenames:
    print f

In [ ]:
#sample specific
toremove = ['ppblsmo18A-IP110-20180702-IP-19000MHz-000.0deg-+3dBm3.0OeRMS-0-21.csv']
for t in toremove:
    filenames.remove(t)
for f in filenames:
    print f

In [ ]:
Hs = []
dPdHs = []
phases = []
freqs = []
for filename in filenames:
    print filename
    H, dPdH, phase = FMR.readFileTruncatePos(filename)
    Hs.append(H)
    dPdHs.append(dPdH)
    phases.append(phase)
    freq = int(filename.partition('MHz')[0].rpartition('-')[-1])/1000.
    freqs.append(freq)

#os.chdir(olddir)

In [ ]:
%matplotlib inline
for i,H in enumerate(Hs):
    print freqs[i]
    plt.plot(H,dPdHs[i])
    plt.show()

In [ ]:
%matplotlib inline
for i,H in enumerate(Hs):
    print freqs[i]
    plt.plot(H,phases[i])
    plt.show()

In [ ]:
%matplotlib inline
dPdHoffsets = []
for i,H in enumerate(Hs):
    dPdH = dPdHs[i]
    #if i > 3:
    fitBG = FMR.subtractBG(H,dPdH)
    print freqs[i]
    plt.show()
    dPdHoffset = dPdH - fitBG
    #else:
    #    dPdHoffset = dPdH
    dPdHoffsets.append(dPdHoffset)

#i = 2
#dPdH = dPdHs[i]
#fitBG = FMR.subtractBG(Hs[i],dPdH,window=10)
#plt.show()
#dPdHoffset = dPdH-fitBG
#dPdHoffsets[i] = dPdHoffset

In [ ]:
NPeaks = 1
fits = []
%matplotlib inline
for i,H in enumerate(Hs):
    #if i > 1 and i < 29:
    dPdHoffset = dPdHoffsets[i]
    guess = FMR.guessN(H,dPdHoffset,NPeaks)
    fit = FMR.fitFMR(H,dPdHoffset,guess)
    fits.append(fit)
    print freqs[i]
    plt.show()

In [ ]:
HFMR = []
errHFMR = []
deltaH = []
errdeltaH = []
phaseLorentzian = []
errphaseLorentzian = []
peakMagnitude = []
errpeakMagnitude = []
for n in np.arange(0,NPeaks):
    peakMagnitude.append(np.array([f[0][0][0+n*4] for f in fits]))
    HFMR.append(np.array([f[0][0][1+n*4] for f in fits]))
    phaseLorentzian.append(np.array([f[0][0][2+n*4] for f in fits]))
    deltaH.append(np.array([f[0][0][3+n*4] for f in fits]))
    errpeakMagnitude.append(np.array([f[1][0][0+n*4] for f in fits]))
    errHFMR.append(np.array([f[1][0][1+n*4] for f in fits]))
    errphaseLorentzian.append(np.array([f[1][0][2+n*4] for f in fits]))
    errdeltaH.append(np.array([f[1][0][3+n*4] for f in fits]))

In [ ]:
HFMRfit = np.polyfit(HFMR[0][10:],freqs[10:],1)
print HFMRfit
Meff = -HFMRfit[1]/HFMRfit[0]
print Meff
#gop = HFMRfit[0]*h/(muB*mu0) #insert h, muB, mu0 in correct units

In [ ]:
%matplotlib inline
for n in np.arange(0,NPeaks):
    plt.errorbar(freqs,HFMR[n],fmt='.',yerr=errHFMR[n])
    plt.plot(np.poly1d(HFMRfit)(HFMR[n]),HFMR[n],'-')
plt.ylabel('HFMR (Oe)')
plt.xlabel('Frequency (Hz)')

In [ ]:
%matplotlib inline
for n in np.arange(0,NPeaks):
    plt.errorbar(freqs,deltaH[n],fmt='.',yerr=errdeltaH[n])
plt.ylabel('FMR Linewidth (Oe)')
plt.xlabel('Frequency (Hz)')

In [ ]:
deltaH[0][0] = -1.*deltaH[0][0]

In [ ]:
%matplotlib inline
for n in np.arange(0,NPeaks):
    plt.errorbar(freqs,phaseLorentzian[n],fmt='.',yerr=errphaseLorentzian[n])
plt.ylabel('Phase (deg)')
plt.xlabel('Frequency (Hz)')

In [ ]:
%matplotlib inline
for n in np.arange(0,NPeaks):
    plt.errorbar(freqs,peakMagnitude[n],fmt='.',yerr=errpeakMagnitude[n])
plt.ylabel('Magnitude (a.u.)')
plt.xlabel('Frequency (Hz)')

In [ ]:
[1]*2